This program will take the composition of coexisting olivine and willemite (expressed as XZn, mole fraction Zn), and use the thermodynamic parameters for DeltaG willemite=olivine for end-member Zn2SiO4 and Mg2SiO4 that have been fit to experimental data to estimate sample pressure or pressure and temperature.

In [1]:
import math
import pandas as pd
import scipy.optimize as optimize

First, calculate DeltaG/RT for each reaction from the composition data. This includes Margules parameters (W) for each phase. This requires a value for temperature.

In [2]:
def olwicompG(ol,wi,Wmg,Wzn,T):
    GRTzn=-math.log(ol/wi)+Wzn/(8.3144598*T)*math.pow((1-wi),2)+Wmg/(8.3144598*T)*math.pow((1-ol),2)
    GRTmg=-math.log((1-ol)/(1-wi))+Wzn/(8.3144598*T)*math.pow((wi),2)+Wmg/(8.3144598*T)*math.pow((ol),2)
    return [GRTzn, GRTmg]

We can test this by calculating DeltaG/RT from EPMA data from the 3 GPa, 1325 °C piston-cylinder experiment. 

In [26]:
print(olwicompG(0.267711892,0.764001815,0,0,1373))

[1.048658792417096, -1.132349912370885]


Now we need a function to find DeltaG/RT at conditions P,T using the Holland & Powell (2011) style model, using the parameters fit to the model by least squares regression. We can use those defined previously.

In [4]:
def HtminusH(phase,T):
    return float(hpdata.loc[phase,'CpA'])*(T-298.15)+0.5*float(hpdata.loc[phase,'CpB'])*(T*T-298.15*298.15)-float(hpdata.loc[phase,'CpC'])*(1/T-1/298.15)+2*float(hpdata.loc[phase,'CpD'])*(math.sqrt(T)-math.sqrt(298.15))

def StminusS(phase,T):
    return 1000*(float(hpdata.loc[phase,'CpA'])*math.log(T/298.15)+float(hpdata.loc[phase,'CpB'])*(T-298.15)-0.5*float(hpdata.loc[phase,'CpC'])*(1/(T*T)-1/(298.15*298.15))-2*float(hpdata.loc[phase,'CpD'])*(1/math.sqrt(T)-1/math.sqrt(298.15)))

def GTP0(phase,T):
    return ((float(hpdata.loc[phase,'H0'])+HtminusH(phase,T))*1000)-(((float(hpdata.loc[phase,'S0']))*1000)+StminusS(phase,T))*T

def EOSa(phase,P):
     return (1+float(hpdata.loc[phase, 'kprime0']))/((1+float(hpdata.loc[phase, 'kprime0'])+(float(hpdata.loc[phase, 'k0'])*float(hpdata.loc[phase, 'kdprime0']))))

def EOSb(phase,P):
     return (float(hpdata.loc[phase, 'kprime0'])/float(hpdata.loc[phase, 'k0']))-float(hpdata.loc[phase, 'kdprime0'])/(1+float(hpdata.loc[phase, 'kprime0']))

def EOSc(phase,P):
     return (1+float(hpdata.loc[phase, 'kprime0'])+float(hpdata.loc[phase, 'k0'])*float(hpdata.loc[phase, 'kdprime0']))/(float(hpdata.loc[phase, 'kprime0'])*float(hpdata.loc[phase, 'kprime0'])+float(hpdata.loc[phase, 'kprime0'])-float(hpdata.loc[phase, 'k0'])*float(hpdata.loc[phase, 'kdprime0']))

def Pth(phase,T):
    theta=10636/(((float(hpdata.loc[phase, 'S0'])*1000)/float(hpdata.loc[phase, 'atoms']))+6.44)
    xi0=(theta/298.15)*(theta/298.15)*math.exp(theta/298.15)/((math.exp(theta/298.15)-1)*(math.exp(theta/298.15)-1))
    Pthermal=float(hpdata.loc[phase, 'alpha0'])*float(hpdata.loc[phase, 'k0'])*theta/xi0*(1/(math.exp(theta/T)-1)-1/(math.exp(theta/298.15)-1))
    return(Pthermal)

def V(phase,P,T):
    return float(hpdata.loc[phase, 'V0'])*(1-EOSa(phase,P)*(1-math.pow((1+EOSb(phase,P)*(P-Pth(phase,T))),(-EOSc(phase,P)))))

def PVcorr(phase,P,T):
    return 1-EOSa(phase,P)+(EOSa(phase,P)*(math.pow((1-EOSb(phase,P)*Pth(phase,T)),(1-EOSc(phase,P)))-math.pow((1+EOSb(phase,P)*(P-Pth(phase,T))),(1-EOSc(phase,P))))/(EOSb(phase,P)*(EOSc(phase,P)-1)*P))

def GPTminusGP0T(phase,P,T):
    return PVcorr(phase,P,T)*P*float(hpdata.loc[phase, 'V0'])

def GPT(phase,P,T):
    return GTP0(phase,T)+GPTminusGP0T(phase,P,T)*1000

# reactant = phase1, product = phase2 

def DeltaGPT(phase1,phase2,P,T):
    return GPT(phase2,P,T)-GPT(phase1,P,T)

def DeltaGPTRT(phase1,phase2,P,T):
    return DeltaGPT(phase1,phase2,P,T)/(8.3144598*T)

For this to work, we should have our thermodynamic parameters in a pandas dataframe called 'hpdata' (can modify this). In the previous script, this was generated by reading the datafile provided by Holland & Powell, but for this example we can use a csv file that contains thermodyamic data for the relevant phases.

In [5]:
hpdata=pd.read_csv('mgzn_thermo_fit.csv', index_col="phase")

# Put S data into kJ/mol rather than J/mol
hpdata.S0=hpdata.S0/1000


Check vales at 1373 K, 1 GPa against the excel spreadsheet:

In [6]:
print(GPT('fo',10,1373))
print(GPT('mgwi',10,1373))
print(GPT('wi',10,1373))
print(GPT('znol',10,1373))

print(DeltaGPT('mgwi','fo',10,1373))
print(DeltaGPTRT('mgwi','fo',10,1373))
print(DeltaGPT('wi','znol',10,1373))
print(DeltaGPTRT('wi','znol',10,1373))

-2409510.8988951133
-2400065.406796808
-1918412.7966396334
-1903410.358794706
-9445.492098305374
-0.8274085682840894
15002.437844927423
1.3141872851992003


In [7]:
def residualolwi(P,T,ol,wi,Wmg,Wzn):
    return [(DeltaGPTRT('wi','znol',P,T)-olwicompG(ol,wi,Wmg,Wzn,T)[0]),(DeltaGPTRT('mgwi','fo',P,T)-olwicompG(ol,wi,Wmg,Wzn,T)[1])]

In [43]:
residualolwi(10,1373,0.267711892,0.764001815,0,0)

[0.2655284927821042, 0.3049413440867955]

In [44]:
def leastsquaresPTolwi(PTguess,ol,wi,Wmg,Wzn):
    def olwisumsq(PTguess,ol,wi,Wmg,Wzn):
        P,T=PTguess
        return math.pow(residualolwi(P,T,ol,wi,Wmg,Wzn)[0],2)+math.pow(residualolwi(P,T,ol,wi,Wmg,Wzn)[1],2)
    return optimize.minimize(olwisumsq, PTguess, args=(ol, wi, Wmg, Wzn), method='Nelder-Mead')

def leastsquareszn(PTguess,ol,wi,Wmg,Wzn):
    def sumsqzn(PTguess,ol,wi,Wmg,Wzn):
        P,T=PTguess
        return math.pow(residualolwi(P,T,ol,wi,Wmg,Wzn)[0],2)
    return optimize.minimize(sumsqzn, PTguess, args=(ol, wi, Wmg, Wzn), method='Nelder-Mead')

def leastsquaresmg(PTguess,ol,wi,Wmg,Wzn):
    def sumsqmg(PTguess,ol,wi,Wmg,Wzn):
        P,T=PTguess
        return math.pow(residualolwi(P,T,ol,wi,Wmg,Wzn)[1],2)
    return optimize.minimize(sumsqmg, PTguess, args=(ol, wi, Wmg, Wzn), method='Nelder-Mead')

In [46]:
PTfitboth=leastsquaresPTolwi([30,1373],0.359663585,0.840529523,0,0)
print(PTfitboth)
PTfitzn=leastsquareszn([30,1373],0.359663585,0.840529523,0,0)
PTfitmg=leastsquaresmg([30,1373],0.359663585,0.840529523,0,0)
print(PTfitzn)
print(PTfitmg)
print('Pressure:', PTfitboth.x[0], 'kbar', 'Temperature', PTfitboth.x[1]-273, '°C')

 final_simplex: (array([[  30.93566261, 1178.92351792],
       [  30.93566953, 1178.92345723],
       [  30.93566271, 1178.92348998]]), array([1.57245014e-15, 3.79308771e-15, 4.98757011e-15]))
           fun: 1.5724501430874928e-15
       message: 'Optimization terminated successfully.'
          nfev: 96
           nit: 49
        status: 0
       success: True
             x: array([  30.93566261, 1178.92351792])
 final_simplex: (array([[  29.64483793, 1188.30775589],
       [  29.64483284, 1188.30779645],
       [  29.64484811, 1188.30767488]]), array([6.02011804e-26, 2.98003165e-25, 3.04952376e-25]))
           fun: 6.020118042484287e-26
       message: 'Optimization terminated successfully.'
          nfev: 135
           nit: 70
        status: 0
       success: True
             x: array([  29.64483793, 1188.30775589])
 final_simplex: (array([[  30.6148603 , 1182.99665709],
       [  30.61486354, 1182.99661557],
       [  30.61486516, 1182.99659479]]), array([4.73809581e-29, 4.0

For the olivine/willemite pressure standard, there is not much good data on temperature dependence on XZn. It is useful to be able to fix temperature at the value recorded by the thermocouple, and to allow only P to be fit.

In [23]:
def leastsquaresPolwi(Pguess,T,ol,wi,Wmg,Wzn):
    def olwisumsq(P,T,ol,wi,Wmg,Wzn):
        return math.pow(residualolwi(P,T,ol,wi,Wmg,Wzn)[0],2)+math.pow(residualolwi(P,T,ol,wi,Wmg,Wzn)[1],2)
    return optimize.minimize(olwisumsq, x0=Pguess, args=(T,ol,wi,Wmg,Wzn))

We can test this against the experimental data for 1325 °C, 30 kbar.

In [25]:
Pfit=leastsquaresPolwi(30,1598,0.387004994,0.894547358,0,0)
print('Pressure:', Pfit.x[0], 'kbar')

Pressure: 14.8974359118015 kbar
